# Tutorial 3 - Boosting Search via Symmetry Breaking, Implied Constraints, Randomisation, and Restarts 

**Important:** Please do not read untill you fully finish the first 2 tutorials.

Congratulations! You are now level one constraint programmer: you know the basics on how to model a problem, how to display solutions, how to evaluate models, and how to choose a good branching strategy! **I'm so proud of you!**

In this tutorial we slowly dive into advanced techniques. We also start to use arithmetic constraints and solve optimisation problems.

In [ ]:
from config import setup
setup()

## 1. Golomb ruler

Your goal is to place $N$ marks on a ruler, such that no two marks are at the same distance and the total length of the ruler (the position of the last mark) is minimized. 

<div class="row" style="margin-top: 10px">
    <img src="display/images/Golomb_Ruler-4.svg" style="display: block; margin: auto; width: 400px;" />
    <p style="margin: auto; margin-top: 10px; text-align: center;">Golomb ruler of order 4 and length 6. This ruler is both optimal and perfect.</p>
</div>

Golomb ruler can be used in information theory to design error correcting codes or in telecommunications to avoid interferences during radio communications. You can read about it here: https://en.wikipedia.org/wiki/Golomb_ruler

<div class="alert alert-block alert-danger">
    
In the rest of this tutorial (except the last part), please use the following parameter when solving your model:

```python
SearchType= "DepthFirst"
```
    
</div>

In order to control the level of filtering (arc consistency, bound consistency, forward checking, etc), CPoptimizer uses a parameter called [`DefaultInferenceLevel`](http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.parameters.py.html?highlight=defaultinferencelevel#docplex.cp.parameters.CpoParameters.DefaultInferenceLevel).

In the rest of this tutorial, you are required to test all three possibilities:

```python
DefaultInferenceLevel=Low
DefaultInferenceLevel=Medium
DefaultInferenceLevel=Extended
```

<div class="alert alert-block alert-info">

After a while, if you see one that you particularly find efficient (runtime), you can use it for the rest of the tutorial.
    
</div>

We are going to create a model for the decision version of this problem, that is, given $n$ marks, and a ruler of size $m$, place the $n$ markers such that no two markers are at the same distance.

You are free to use any constraint you want. However, you must declare and use the minimum ammount of constraints (**NOT A SINGLE UNNESSASARY CONSTRAINT**).

<div class="alert alert-block alert-success">
    
Note that for N marks, a ruler of length $2 ^ {N -1}$ can be found (I let you figure out why).
    
</div>

**Exercice:** Write a funtion `decision_model(n, m)` that builds and returns the corresponding model. 

In [ ]:
from docplex.cp.model import CpoModel

def decision_model(n: int, m: int) -> CpoModel:
    ...  # TODO

**Exercice:** Solve the problem for $n=4$ and $m=6$. Then try different values of $n$ and $m$ (but do not waste too much time). 

<div class="alert alert-block alert-info">

You can display the solution using:
    
```python
from display import golomb as display_golomb

# marks is the list of variables
display_golomb([sol[m] for m in marks])
```
    
</div>

**Exercice:** Print and display all the sulutions for $(n, m) = (4, 6)$ and $(4,7)$.

**Exercice:** Write a funtion `basic_optimisation_model(n)` that builds and returns the corresponding model for the
optimisation problem. Note that an optimisation function can be seen as a variable. In order to specify the variable to optimise, we can simply use: 

```python
model.add(model.minimize(myvariable))
```

or 

```python
model.add(model.maximize(myvariable))
```


In [ ]:
def basic_optimisation_model(n: int) -> CpoModel:
    ...  # TODO

**Exercice:** Solve the optimisation problem for $n = 6, \ldots{}, 10$  and display the solutions.

## 2. Symmetry Breaking

In combinatorial optimisation, two (partial) solutions are called symmetric if we can find a transformation from one to the other. 
Consider our golomb ruler problem. Given any solution to the marks variables, if the first mark is not at index $0$, we can always shift everything to the left to start from $0$ and still have a solution. 

Constraint programming is extremely flexible to handle symmetries since they can be declared as constraints. 

In the case of the above symmetry, we can simply add a constraint to force the first mark to be at position `0`:

```python
model.add (marks[0]==0)
```

**Exercice:** This problem has another symmetry, can you find it? In order to help you, display the solution for $n=4$ and $m=6$ for the decision problem. You should find 2 solutions that are essentially the same. Can you find the symmetry? How can we model this symmetry as a constraint? 

**Exercice:** Write a new function `nosymmetry_optimisation_model(n)` that builds a noew model that avoids the two symmetries we found so far. 

In [ ]:
def nosymmetry_optimisation_model(n: int) -> CpoModel:
    ...  # TODO

**Exercice:** Compare `nosymmetry_optimisation_model` and `basic_optimisation_model` for different values of $n$ (you decide the values of $n$). Plot the runtime and the search tree size.

**Question:** What is your impression about symmetries? 

## 3. Implied Constraints

An implied constraint is one that can be dedused by looking at the original constraints of the problem. 
For instance, if we have $a<b $ and $b<c$, one can infer that $a<c$. 

Such constraints (called also redundant constraints) can help the solver to prune further the search tree. 

**Question:** In our problem there is one implied constraint. Can you find it? Please check with of the supervisors. 

**Exercice:** Write a new function `nosymmetry2_optimisation_model(n)` that adds the implied constraint to the `nosymmetry_optimisation_model(n)` and returns the new model.

In [ ]:
def nosymmetry2_optimisation_model(n: int) -> CpoModel:
    ...  # TODO

**Exercice:** Compare `nosymmetry2_optimisation_model` and `nosymmetry_optimisation_model`. 

## 4. Randomisation and Restarts

**Exercice:** Declare two search strategies: one that uses a lexicographical order on both variables and values, 
and the other using an impact-based choice on the variables with a random value selection.

**Exercice:** Run the two strategies using the `nosymmetry2_optimisation_model` for different values of $n$.

Combinatorial search exhibits usually a bad behaviour in the runtime distribution called **heavy tailed phenomenon**. 
That is, at any node of the search tree, there is a non-negligeable probability that the time needed to explore the current subtree is heavier than 
an exponential distribution (you can read about it here https://aaai.org/Papers/AAAI/1998/AAAI98-061.pdf. 


A simple solution to deal with such a bad behaviour is to restart search from time to time. 
CPOptimizer offers this choice by using the parameter: 
    
```python
SearchType="Restart"
```

**Exercice:** Using a restart search, evaluate the two strategies mentionned above using the `nosymmetry2_optimisation_model` for different values of $n$. What can you conclude?

**Exercice:** What is the maximum value of $n$ for which you can solve this problem? Use all your techniques! 

## 5. Conclusion

**Question:** What did you learn today? 

<div class="alert alert-block alert-danger"></div>